In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import bs4
import re
from time import sleep
import os
import pymorphy2
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from tqdm import tqdm
import pickle
from scipy.sparse import csr_matrix
import gensim
from gensim.models import Doc2Vec
from multiprocessing import Pool, Lock, Value
import functools
import operator

In [2]:

morph = pymorphy2.MorphAnalyzer()


In [ ]:
stop_words = set(stopwords.words('russian'))


In [6]:
with open('ordered_docks.pickle','rb') as f:
    ma = pickle.load(f)

In [7]:
ma_li = list(ma)

In [11]:
words_h = dict()
words_b = dict() #глобальный словарь слов
ind_h = 0
ind_b = 0
y_h_ind = []
x_h_ind = []
x_b_ind = []
y_b_ind = []
data_h = []
data_b = []
i = 0
tot_length = 0
for k in tqdm(ma_li):
    with open('text-relevance-new/content/content/'+k+'.txt') as f:     
        terms = list()
        for term in f:    
            if term[:-1] not in stop_words:
                terms.append(term[:-1]) 
        seen_h = set()
        seen_b = set()
        fl = 0
        curs = dict()
        curs_h = dict()
        for term in terms:
            if fl == 0:
                if term == 'STOP_ITS_NOT_HEADER':
                    fl = 1
                    continue
                if term not in seen_h:
                    seen_h.add(term)
                    curs_h[term] = len(data_h)
                    data_h.append(1)
                    x_h_ind.append(i)
                    if words_h.get(term, -1) == -1:
                        words_h[term] = ind_h
                        ind_h += 1
                    y_h_ind.append(words_h[term])
                else:
                    data_h[curs_h[term]] += 1    
            else :
                    if term not in seen_b:
                        seen_b.add(term)
                        curs[term] = len(data_b)
                        data_b.append(1)
                        x_b_ind.append(i)
                        if words_b.get(term, -1) == -1:
                            words_b[term] = ind_b
                            ind_b += 1
                        y_b_ind.append(words_b[term])
                    else:
                        data_b[curs[term]] += 1
        i += 1    

100%|██████████| 38114/38114 [14:37<00:00, 43.44it/s]  


In [26]:
class Generator(object):
    def __iter__(self):
        for k in tqdm(ma_li):
            with open('text-relevance-new/content/content/'+k+'.txt') as f:
                terms = list()
                for term in f:
                    if term[:-1] not in stop_words:
                        terms.append(term[:-1])
            yield terms

In [28]:
w2v_model = Word2Vec(min_count=19,
                     window=7,
                     size=250,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=6)

In [29]:
gen = Generator()
w2v_model.build_vocab(gen)

100%|██████████| 38114/38114 [04:05<00:00, 154.98it/s]


In [31]:
w2v_model.train(gen, total_examples=w2v_model.corpus_count, epochs=1)

100%|██████████| 38114/38114 [08:48<00:00, 72.10it/s] 


(119957291, 308320688)

In [16]:
sp_h = csr_matrix((data_h, (x_h_ind, y_h_ind)), shape=(i, ind_h),dtype = numpy.int32) 
sp_b = csr_matrix((data_b, (x_b_ind, y_b_ind)), shape=(i, ind_b),dtype = numpy.int32)
    

NameError: name 'csr_matrix' is not defined

In [26]:
# coding: UTF-8
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import scipy.sparse as sp
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.feature_extraction.text import _document_frequency

class BM25Transformer(BaseEstimator, TransformerMixin):
    """
    Parameters
    ----------
    use_idf : boolean, optional (default=True)
    k1 : float, optional (default=2.0)
    b : float, optional (default=0.75)
    References
    ----------
    Okapi BM25: a non-binary model - Introduction to Information Retrieval
    http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html
    """
    def __init__(self, use_idf=True, k1=2.0, b=0.75):
        self.use_idf = use_idf
        self.k1 = k1
        self.b = b

    def fit(self, X):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            document-term matrix
        """
        if not sp.issparse(X):
            X = sp.csc_matrix(X)
        if self.use_idf:
            n_samples, n_features = X.shape
            df = _document_frequency(X)
            idf =   np.log((n_samples - df + 0.5) / (df + 0.5))
            self._idf_diag = sp.spdiags(idf, diags=0, m=n_features, n=n_features)
        return self

    def transform(self, X, copy=True):
        """
        Parameters
        ----------
        X : sparse matrix, [n_samples, n_features]
            document-term matrix
        copy : boolean, optional (default=True)
        """
        if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
            # preserve float family dtype
            X = sp.csr_matrix(X, copy=copy)
        else:
            # convert counts or binary occurrences to floats
            X = sp.csr_matrix(X, dtype=np.float64, copy=copy)

        n_samples, n_features = X.shape

        # Document length (number of terms) in each row
        # Shape is (n_samples, 1)
        dl = X.sum(axis=1)
        # Number of non-zero elements in each row
        # Shape is (n_samples, )
        sz = X.indptr[1:] - X.indptr[0:-1]
        # In each row, repeat `dl` for `sz` times
        # Shape is (sum(sz), )
        # Example
        # -------
        # dl = [4, 5, 6]
        # sz = [1, 2, 3]
        # rep = [4, 5, 5, 6, 6, 6]
        rep = np.repeat(np.asarray(dl), sz)
        # Average document length
        # Scalar value
        avgdl = np.average(dl)
        # Compute BM25 score only for non-zero elements
        print(np.max(X.data))
        data = X.data * (self.k1 + 1) / (X.data + self.k1 * (1 - self.b + self.b * rep / avgdl))
        data[data < 0] = 0
        X = sp.csr_matrix((data, X.indices, X.indptr), shape=X.shape)

        if self.use_idf:
            check_is_fitted(self, '_idf_diag', 'idf vector is not fitted')

            expected_n_features = self._idf_diag.shape[0]
            if n_features != expected_n_features:
                raise ValueError("Input has n_features=%d while the model"
                                 " has been trained with n_features=%d" % (
                                     n_features, expected_n_features))
            # *= doesn't work
            X = X * self._idf_diag

        return X

In [27]:
trans = BM25Transformer()

In [28]:
trans.fit(sp_b)

BM25Transformer(b=0.75, k1=2.0, use_idf=True)

In [30]:
sp_b_t = trans.transform(sp_b)

/home/sargeras/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.


78662.0


In [31]:
trans.fit(sp_h)

BM25Transformer(b=0.75, k1=2.0, use_idf=True)

In [32]:
sp_h_t = trans.transform(sp_h)

87.0


/home/sargeras/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:50: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.


In [33]:
def easy_tokenizer(text):
    word = str()
    for symbol in text:
        if symbol.isalnum(): word += symbol
        elif word:
            yield word
            word = str()
    if word: yield word

PYMORPHY_CACHE = {}
MORPH = None
#hint, чтобы установка pymorphy2 не была бы обязательной
def get_lemmatizer():
    global MORPH
    if MORPH is None: MORPH = pymorphy2.MorphAnalyzer()
    return MORPH

def pymorphy_tokenizer(text):
    global PYMORPHY_CACHE
    for word in easy_tokenizer(text):
        word_hash = hash(word)
        if word_hash not in PYMORPHY_CACHE:
            PYMORPHY_CACHE[word_hash] = get_lemmatizer().parse(word)[0].normal_form          
        yield PYMORPHY_CACHE[word_hash]

In [34]:
queries = pd.read_csv('fix4.txt', sep='\t', header=None)
urls = pd.read_csv('urls.txt', sep='\t', header=None)

In [36]:
sor_docs =  pd.read_csv('text_relevance1.csv')

In [39]:
queries = queries.values.tolist()


In [42]:
qu_split = []
for g in range(399):
    words = pymorphy_tokenizer(queries[g][0])
    word = []
    for k in words:
        if k not in stop_words:
            word.append(k) 
    qu_split.append(word)

In [143]:
with open("sevenwindows.pickle",'rb') as f:
    w2v_model = pickle.load(f)

In [133]:
sample = pd.read_csv('sample.technosphere.ir1.textrelevance.submission.txt')

In [166]:
def res_c1(g):
        yy = np.array(sample[sample['QueryId'] == g+1]['DocumentId'].values.tolist()) - 1

        t =  csr_matrix((len(yy), 1), dtype=np.float64)
        for w in qu_split[g]:
            if words_h.get(w, -1) != -1:
                t += sp_h_t[yy,words_h[w]]
            
            if words_b.get(w, -1) != -1:
                t += sp_b_t[yy,words_b[w]]           
        s = 0
        for w in qu_split[g]:
            if w2v_model.wv.vocab.get(w,-1) == -1:
                s = 1
                break
        if s == 0:
            k_b = 1
            k_h = 1
            for a in w2v_model.wv.most_similar_cosmul(positive=qu_split[g]):
                POS = morph.parse(a[0])[0].tag.POS
                gg = morph.parse(a[0])[0].tag.cyr_repr
                if not ((POS == 'NOUN' or POS is None) and gg.find('имя') == -1 and gg.find('фам') == -1 \
                       and gg.find('гео') == -1):
                     continue
                if a[1] > 0.6:
                    if words_b.get(a[0], -1) != -1:
                        t += (a[1] / k_b) *sp_b_t[yy,words_b[a[0]]]
                        k_b += 1
                        
                    if words_h.get(a[0], -1) != -1:
                        t += (a[1] / k_h) * sp_h_t[yy,words_h[a[0]]]   
                        k_h += 1
             
        t = t.toarray().reshape(-1)
        o = np.argsort(t)[-10:]
        return   yy[o] + 1

In [170]:
mutex = Lock()
n_processed = Value('i', 0)

def func_wrapper(g):
    res = res_c1(g)

    with mutex:
        global n_processed
        n_processed.value += 1
        print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=6) as pool:
    ans = pool.map(func_wrapper, range(399))

нужно
химический
допустимый
вернуть
край
путём
обменивать
налог
прилагать
выменивать
доказать
земля
присутствие
добавлять
наличие
аэропорт
менять
страна
прибавлять
сульфат
масло
побывать
барий
механический
жидкость
посетить
твёрдый
коробка
карбонат
передача
барий
1 objects are processed...сделать
3 objects are processed...написать
4 objects are processed...распоряжаться
господствовать
добраться
руководить
носить
таскать
5 objects are processed...царить
деревня
главенствовать
6 objects are processed...операция
контактный
подключить
управлять
лик
править
возглавлять
лицо
заведовать
передний
командовать
выход
область
9 objects are processed...день
написать
почему
свет
10 objects are processed...вода
оборот
11 objects are processed...пить
спина
цикл
ставить
почка
употреблять
сдавать
пускать в ход
прибегать
эксплуатировать
утилизировать
пользоваться
применять
использовать
12 objects are processed...лечить
пиявка
врачевать
13 objects are processed...пользовать
муравей
огород
язык
народный
ди

счастие
счастье
некоторый
мужчина
мужик
282 objects are processed...звать
дядя
здоровый
283 objects are processed...крепкий
сколько
284 objects are processed...сделать
мощный
километр
285 objects are processed...учебный
дюжий
мода
286 objects are processed...планирование
поветрие
сильный
й
287 objects are processed...приготовить
район
поплавок
русский
россиянин
машина
неприятный
унитаз
белый друг
аппарат
запах
дух
белый телефон
288 objects are processed...гинеколог
сразу
определить
беременность
289 objects are processed...узнать
выяснить
есть
питаться
292 objects are processed...устанавливать
сколько
сделать
телефон
время
293 objects are processed...идти
лавка
магазин
294 objects are processed...необходимый
нужный
пополнение
крещение
дополнение
дитя
чадо
документ
ребёнок
командировка
автотранспорт
295 objects are processed...хороший
296 objects are processed...малютка
297 objects are processed...крошка
провести
ребёнок
узнать
новорождённый
дитя
выяснить
младенец
пошлина
298 objects are

In [172]:
for u in range(399):
    ans[u] = ans[u][::-1]

In [174]:
qu_id = []
doc_rel = []

In [175]:
for y in range(399 * 10):
    qu_id.append(y // 10 + 1)
    doc_rel.append(ans[y // 10][y % 10])

In [176]:
result_df = pd.DataFrame({
    'QueryId' : qu_id,
    'DocumentId' : doc_rel
})

In [178]:
result_df.to_csv('tomorrow.txt', index=False)